## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "Theodoros Ioannidis"
AEM = "3626"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [1]:
# Always run this cell
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neural_network import MLPClassifier
from statistics import mean
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error, balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier, StackingClassifier, RandomForestRegressor, BaggingRegressor, VotingClassifier, AdaBoostRegressor, GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [2]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7faa1ccddbd0>)

In [3]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values
test_set = pd.read_csv("test_set_noclass.csv").sample(frac=1).reset_index(drop=True)

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is possible to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [ ]:
from numpy.core.numeric import cross
# BEGIN CODE HERE

cls1 = KNeighborsClassifier(n_neighbors=8)
cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10) #8
cls3 = LogisticRegression(penalty='l2',random_state=RANDOM_STATE, n_jobs=-1)
models = [('KNN', cls1), ('DT', cls2), ('LG', cls3)]
soft_vcls = VotingClassifier(estimators=models, n_jobs=-1, voting='soft') # Voting Classifier
hard_vcls = VotingClassifier(estimators=models, n_jobs=-1, voting='hard') # Voting Classifier
svlcs_scores = cross_validate(soft_vcls, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
s_avg_fmeasure = svlcs_scores['test_f1_weighted'].mean() # The average f-measure
s_avg_accuracy = svlcs_scores['test_balanced_accuracy'].mean() # The average accuracy

hvlcs_scores = cross_validate(hard_vcls, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
h_avg_fmeasure = hvlcs_scores['test_f1_weighted'].mean() # The average f-measure # The average f-measure
h_avg_accuracy = hvlcs_scores['test_balanced_accuracy'].mean() # The average accuracy
#END CODE HERE

In [ ]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=8)),
                             ('DT',
                              DecisionTreeClassifier(min_samples_leaf=10,
                                                     random_state=42)),
                             ('LG',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.8286 & Balanced Accuracy: 0.8202


You should achive above 82% (Soft Voting Classifier)

In [ ]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=8)),
                             ('DT',
                              DecisionTreeClassifier(min_samples_leaf=10,
                                                     random_state=42)),
                             ('LG',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8246 & Balanced Accuracy: 0.8203


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [ ]:
# BEGIN CODE HERE
cls1 = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1) # Classifier #1 
cls2 = GradientBoostingClassifier(n_estimators=300, max_depth = 5, random_state=RANDOM_STATE) # Classifier #2 
cls3 = MLPClassifier() # Classifier #3 (Optional)
estimators = [('RF', cls1), ('GB', cls2), ('MLP', cls3)]
scls = StackingClassifier(estimators, LogisticRegression(random_state=RANDOM_STATE)) # Stacking Classifier
scores = cross_validate(scls, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
avg_fmeasure = scores['test_f1_weighted'].mean() # The average f-measure
avg_accuracy = scores['test_balanced_accuracy'].mean() # The average accuracy
#END CODE HERE

In [ ]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('RF',
                                RandomForestClassifier(n_estimators=300,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('GB',
                                GradientBoostingClassifier(max_depth=5,
                                                           n_estimators=300,
                                                           random_state=42)),
                               ('MLP', MLPClassifier())],
                   final_estimator=LogisticRegression(random_state=42))
F1 Weighted Score: 0.8584 & Balanced Accuracy: 0.851


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [ ]:
# BEGIN CODE HERE
cls1 = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1) # Classifier #1 
cls2 =  KNeighborsClassifier()# Classifier #2 
cls3 = LogisticRegression(random_state=RANDOM_STATE) # Classifier #3 (Optional)
estimators = [('RF', cls1), ('KNN', cls2)]
ens1 = StackingClassifier(estimators, LogisticRegression(random_state=RANDOM_STATE),n_jobs=-1)
ens2 = BaggingClassifier(DecisionTreeClassifier(max_depth= 5), n_jobs=-1, n_estimators=300, max_samples = 0.2, max_features = 0.5, random_state=RANDOM_STATE)
ens3 = GradientBoostingClassifier(random_state=RANDOM_STATE)
tree = DecisionTreeClassifier()
ens1_scores = cross_validate(ens1, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
ens2_scores = cross_validate(ens2, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
ens3_scores = cross_validate(ens3, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
tree_scores = cross_validate(tree, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
f_measures = {'Stacking':ens1_scores['test_f1_weighted'].mean(), 'Bagging':ens2_scores['test_f1_weighted'].mean(), 'Gradient Boosting':ens3_scores['test_f1_weighted'].mean(), 'tree':tree_scores['test_f1_weighted'].mean()}
accuracies = {'Stacking':ens1_scores['test_balanced_accuracy'].mean(), 'Bagging':ens2_scores['test_balanced_accuracy'].mean(), 'Gradient Boosting':ens3_scores['test_balanced_accuracy'].mean(), 'tree':tree_scores['test_balanced_accuracy'].mean()}
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}

#END CODE HERE

In [ ]:
#Grid Search(IGNORE)
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5],
    'n_estimators' : [80, 100, 150, 200, 300]
}

clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(), max_features = 0.5, n_jobs=-1), param_grid)
clf.fit(X,y)
print(" Results from Grid Search " )
print("\n The best parameters across ALL searched params:\n",clf.best_params_)

 Results from Grid Search 

 The best parameters across ALL searched params:
 {'base_estimator__max_depth': 5, 'max_samples': 0.2, 'n_estimators': 300}


In [ ]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

StackingClassifier(estimators=[('RF',
                                RandomForestClassifier(n_jobs=-1,
                                                       random_state=42)),
                               ('KNN', KNeighborsClassifier())],
                   final_estimator=LogisticRegression(random_state=42),
                   n_jobs=-1)
BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                  max_features=0.5, max_samples=0.2, n_estimators=300,
                  n_jobs=-1, random_state=42)
GradientBoostingClassifier(random_state=42)
DecisionTreeClassifier()
Classifier:Stacking -  F1 Weighted:0.8223
Classifier:Bagging -  F1 Weighted:0.771
Classifier:Gradient Boosting -  F1 Weighted:0.8247
Classifier:tree -  F1 Weighted:0.7008
Classifier:Stacking -  BalancedAccuracy:0.8123
Classifier:Bagging -  BalancedAccuracy:0.7516
Classifier:Gradient Boosting -  BalancedAccuracy:0.8145
Classifier:tree -  BalancedAccuracy:0.6917


**2.2** Describe your classifiers and your results.

YOUR ANSWER HERE

Έπελεξα έναν classifier Boosting, έναν Stacking κι έναν Bagging, με ένα Decision Tree, ώστε να καλυφθούν και οι 3 διαφορετικοί μέθοδοι δημιουργίας ensemble. Σαν δένδρο επιλέχθηκε ένα απλό Decision Tree. Στο stacking χρησιμοποίησα έναν classifier random forest κι έναν K Neighbors, μαζί με έναν  Όλα τα μοντέλα έβγαλαν σημαντικά καλύτερα αποτελέσματα από το Decision Tree, με καλύτερα όλων να είναι το Stacking και το Gradient Boosting. Για την επιλογή ορισμάτων στον Bagging Classifier χρησιμοποιήθηκε Grid Search.

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

Η αύξηση των estimators θα βελτιώσει την ακρίβεια του μοντέλου, ωστόσο θα αυξήσει δραματικά τον χρόνο εκτέλεσης. Για να επιλυθεί αυτό, μία καλή πρακτική θα ήταν να ορίσουμε αρκετά χαμηλό max_depth στο δένδρο που χρησιμοποείται, ώστε να μειωθεί ο χρόνος εκτέλεσης του εκάστοτε δένδρου, με συνέπεια να μειωθεί ο χρόνος εκτέλεσης του bagging, όπως και να ορίσουμε max features και samples. Στην περίπτωση του Boosting ναι μεν θα μπορούσαμε να εφαρμόσουμε παρόμια τακτική, αλλά επειδή ήδη το προκαθορισμένο max depth του Gradient Boosting είναι πολύ χαμηλό(3), θα είχε περισσότερο νόημα να παίξουμε με τα min_samples_split και min_samples_leaf.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [4]:
# BEGIN CODE HERE
cls1 = RandomForestClassifier(n_estimators=350, random_state=RANDOM_STATE, n_jobs=-1)
cls2 = GradientBoostingClassifier(n_estimators=350, max_depth = 6, random_state=RANDOM_STATE)
cls3 = MLPClassifier()
estimators = [('RF', cls1), ('GB', cls2), ('MLP', cls3)]
best_cls = StackingClassifier(estimators, LogisticRegression(random_state=RANDOM_STATE))
scores = cross_validate(best_cls, X=X, y=y, scoring=['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
best_fmeasure = scores['test_f1_weighted'].mean()
best_accuracy = scores['test_balanced_accuracy'].mean()
#END CODE HERE

In [5]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:0.8588082038997055 & Balanced Accuracy:0.8514502036994799


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

YOUR ANSWER HERE

Μετά τους ελέγχους που έγιναν στα προηγούμενα ερωτήματα, αποφάσισα ότι ο βέλτιστος classifier ήταν stacking, με Random Forest, Gradient Boosting και MLP classifiers, με την διαφορά ότι αυτήν την φορά αύξησα τους estimators των Random Forest και Gradient Boosting από 300 σε 350 κι αύξησα το max depth του Gradient Boosting από 4 σε 5. Τα αποτελέσματα, όπως και την πρώτη φορά που εκτελέστηκαν πάνω, ήταν αρκετά καλά, ξεπερνώντας το 85%.

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [6]:
# BEGIN CODE HERE
cls = best_cls.
cls.fit(X,y)
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!